Goal, to be able to take the approximator fn class, data and learning algol and predict learnability.

Learnability defined as? How `often`(from random inits) a `good`(low loss on test set?) `solution`(converged?) is learnt?

- If $\mathcal C_{data} > \mathcal C_{model}$ then not learnable.
- If $\mathcal C_{data} < \mathcal C_{model}$ then learnable (but might overfit?).
- If $\mathcal C_{data} << \mathcal C_{model}$ then not learnable (hmm, empirically not true...).


Note; we only get to control the model, thus the focus on representional capacity measures?

OK. So the goal is. We measure the `complexity` of the data in various ways, and then correlate them with `learnability` (in this case, various models and optimisers?).

## Representational complexity (/Capacity measures)

Capacity measures allow us to cheat? As we use some form of Occam's razor assumption. 
The 'simplest' function that can fit the data will generalise the best.
BUT, this is complicated by the fact that 'fit' may not be perfect. Thus we may need to trade-off fit and simplicity.

Will try some alternatives? 
- A couple of NN architectures (will need to be scale according to size of data???)
- Boosting algols? Xgboost? Maybe using decision trees?
- SVM?

## Data complexity

How can this be measured?
- Data dimension/manifold dim?
- Topology of decision boundaries (for classifiers)
- Moments of the data?
- Smoothness?
- how close decision boundaries are to each other.
- Persistent homology. Barcode diagrams. Topology of NN networks formed by expanding balls around the data points.
- if there is a train and test set then we would like to measure the difference.
- 



On Characterizing the Capacity of Neural Networks using Algebraic Topology

Is how the data arrives (ie online, delayed, ...) a problem within data complexity or within the optimisation algol?


#### VC dimension

What are they up to with the VC dimension? 
Goal is to be able to match representational capacity with complexity in the data. So, the VC dimension allows us to ask whether the function class we are learning from is able to shattern the data we have been given.

If we can measure both then we might give accurate predictions of how learning will proceed.

But, it is quite broken. Just being able to shattern something doesnt mean that it is possible to learn that shatterning. It doesnt take into account any learning dynamics.

So a lower VC dimension means less possibly functions are representable. Thus(?) making it harder to overfit.

A measure of flexibility (but also complexity).
Can the fn shatter all assignments of labels. Is it flexible enough to move decision boundaries around as required.
Relatedly, is it complex enough to contort the decision boundary as required.

In [1]:
import tensorflow as tf

# want a set of different data. measure their complexity
# use UCI datasets and tensorflow plug-in?
# https://www.kaggle.com/uciml/datasets
# https://github.com/JackDunnNZ/uci-data

In [155]:
import os
import numpy as np
import pandas as pd
import scipy as sci
import scipy.stats

In [25]:
basedir = '/home/act65/.kaggle/datasets/uciml/'

In [135]:
def get_data_paths(basedir):
    datafiles = []
    for root, dirs, files in os.walk(basedir):
    #     print(root, dirs, files)
        for f in files:
            if f.endswith('.csv'):
                if 'assay' not in root:
                    datafiles.append(os.path.join(root, f))
    return datafiles

fnames = get_data_paths(basedir)
print(len(fnames))

20


In [137]:
train = [f for f in fnames if 'train' in f]
test = [f for f in fnames if 'test' in f]
print(len(train), len(test))
train, test

3 3


(['/home/act65/.kaggle/datasets/uciml/youtube-comedy-slam/comedy_comparisons-train.csv',
  '/home/act65/.kaggle/datasets/uciml/aps-failure-at-scania-trucks-data-set/aps_failure_training_set_processed_8bit.csv',
  '/home/act65/.kaggle/datasets/uciml/aps-failure-at-scania-trucks-data-set/aps_failure_training_set.csv'],
 ['/home/act65/.kaggle/datasets/uciml/youtube-comedy-slam/comedy_comparisons-test.csv',
  '/home/act65/.kaggle/datasets/uciml/aps-failure-at-scania-trucks-data-set/aps_failure_test_set_processed_8bit.csv',
  '/home/act65/.kaggle/datasets/uciml/aps-failure-at-scania-trucks-data-set/aps_failure_test_set.csv'])

In [111]:
datasets = [pd.read_csv(f) for f in fnames]
ds = datasets[-1]

In [121]:
def hash_categorical(data):
    mapping = dict(zip(set(data), range(len(data))))
    # returns {'category1': 0, 'category2': 1, ...}
    return [mapping[d] for d in data]

def cast_strings(ds):
    for col in ds.columns.values:
        if ds[col].dtype not in ['int64', 'float64']:
            # probably want to keep track of which cols were categorical??
            ds[col] = hash_categorical(ds[col])
    return ds

ds = cast_strings(ds)
arr = np.array(ds)  # could use tf from here.

In [126]:
arrays = [np.array(cast_strings(ds), dtype=np.float32) for ds in datasets]

In [127]:
print([(arr.shape, arr.dtype) for arr in arrays])

[((916, 2), dtype('float32')), ((130, 21), dtype('float32')), ((2339, 3), dtype('float32')), ((177, 2), dtype('float32')), ((138, 19), dtype('float32')), ((1314, 2), dtype('float32')), ((1161, 5), dtype('float32')), ((702, 2), dtype('float32')), ((330, 2), dtype('float32')), ((225592, 3), dtype('float32')), ((912968, 3), dtype('float32')), ((536, 10), dtype('float32')), ((1599, 12), dtype('float32')), ((16000, 171), dtype('float32')), ((16000, 171), dtype('float32')), ((60000, 171), dtype('float32')), ((60000, 171), dtype('float32')), ((310, 7), dtype('float32')), ((310, 7), dtype('float32')), ((583, 11), dtype('float32'))]


In [138]:
def parse_dataset(f):
    df = pd.read_csv(f)
    df = cast_strings(df)
    return np.array(df, dtype=np.float32)

In [140]:
TRAIN = [parse_dataset(f) for f in train]

In [141]:
TEST = [parse_dataset(f) for f in test]

In [143]:
A = TRAIN[0]
B = TEST[0]

In [144]:
A.shape, B.shape

((912968, 3), (225592, 3))

In [162]:
print(np.mean(A, axis=0), np.mean(B, axis=0))
np.mean(A, axis=0) - np.mean(B, axis=0)

[8.226817e+03 8.211416e+03 4.823422e-01] [1.9323999e+03 1.9287136e+03 4.8382035e-01]


array([ 6.2944175e+03,  6.2827021e+03, -1.4781356e-03], dtype=float32)

In [152]:
print(np.var(A, axis=0), np.var(B, axis=0))
np.var(A, axis=0) - np.var(B, axis=0)

[2.0784928e+07 2.0876456e+07 2.4968819e-01] [1.3218791e+06 1.3093082e+06 2.4973819e-01]


array([ 1.9463048e+07,  1.9567148e+07, -4.9993396e-05], dtype=float32)

In [164]:
sci.stats.moment(A, 2, axis=0)

array([2.0784928e+07, 2.0876456e+07, 2.4968819e-01], dtype=float32)

## Graph representations

Need a way to embed tensorflow graphs?!
Rather than using hand designed features such as; the number of params, depth, width, nonlinearity, ... it would  make more sense to learn which features are important for learnability.

In [ ]:
def read_graph():
    pass

def embed_graph():
    pass

## Meta learning

Would be cool. To use all the data generated. Model hyperparams, data complexity and learnability to train a metalearner that predicts/optimises learnability. We could then try to interpret what it has learned!?

Define learnability as ???


E.g. 
3 x lr, 3 x init, 3 x batch = 27 runs. Stop when learning has slowed to ??.

- Var(runs) = how reliably can it be learned? (a function of )
- Mean(runs) = how well can it be learned? (a function of the representation? aaaand the optimisation, and the data sampled...)
- Mean(n_iters of each run?) = how much does it cost to learn?
- Var(h_params) = how easy it is to find a good setting of hparams...
- ?

Learnability = easy + quality + cost?
Or just predict each seprarately?

Ideally want a dataset of 10,000. That would require, say, 100 datasets, and 100 function approximators. Giving a total of 10,000 * 27 runs. If each takes approx 5 min (optimistically), then that totals nearly 100 days of compute time...



#### Leveraging existing data?

Algorithms that work on given problems are the therorists data. 
Dataset; 
`(approximator, optimiser, data, loss)`
* (AlexNet, SGD, ImageNet, Accuracy - 0.95)
* (LeNet, ImageNet, Accuracy - 0.96)
* (...)
* (Seq2seq w atten, translation dataset, BLEU - ?)
* 

Aka, meta learning...? So, lets go create our own dataset...